In [16]:
import os
import json
import keyring
import requests
import traceback
import pandas as pd
from OsOps import Ops
from spotipy import client, util
from collections import namedtuple

In [ ]:
# ADD "GET LATEST PODCASTS"

In [17]:
# AUTHENTICATE (POST)
# !! REQUIRES CREDS STORED IN WIN CRED MGR

# !! ADD CHECK IF NOT STORED

user_id = keyring.get_password("spottyPie_user_id", "")
sp_cid = keyring.get_password("spottyPie_clientID", "")
sp_sec = keyring.get_password("spottyPie_clientSecret", "")

In [42]:

# Handling exceptions (split to module)
#   THEN ADD RESPONSE-BASED ACTION ( eg 200, 400/404 )


try: sp_auth_resp = requests.post( 'https://accouhonkken', {
        'grant_type': 'client_credentials',
        'client_id': sp_cid,
        'client_secret': sp_sec, })
except Exception as exc: print( 
    f"[ ERRR ] {exc.__class__}"
    f"\n[ DSTR ] {exc.__doc__}"
    f"\n[ CTXT ] {exc.__context__}"
    f"\n{ '='*79 }"
    f"\n{traceback.format_exc()}" )
else: print( sp_auth_resp )


[ ERRR ] <class 'requests.exceptions.ConnectionError'>
[ DSTR ] A Connection error occurred.
[ CTXT ] HTTPSConnectionPool(host='accouhonkken', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000024038626AA0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Traceback (most recent call last):
  File "C:\Users\KOJIN\AppData\Roaming\Python\Python310\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\KOJIN\AppData\Roaming\Python\Python310\site-packages\urllib3\util\connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "c:\Program Files\Python310\lib\socket.py", line 955, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, a

In [ ]:
ops = Ops()

def auth_SpotPy():
    """auths both api directly and a Spotipy object"""
    sp_auth_url = 'https://accounts.spotify.com/api/token'

    sp_auth_resp = requests.post( sp_auth_url, {
        'grant_type': 'client_credentials',
        'client_id': sp_cid,
        'client_secret': sp_sec, })
    
    if sp_auth_resp 
    
    sp_auth_resp_json = sp_auth_resp.json()
    sp_auth_token = sp_auth_resp_json['access_token']
    
    token = util.prompt_for_user_token(
        username= user_id, 
        scope= " ".join( [
            "playlist-read-private",
            "playlist-modify-private",
            "user-library-modify",
            "user-read-private" ]),
        client_id= sp_cid, 
        client_secret= sp_sec, 
        redirect_uri = "http://example.com/")

    return client.Spotify(token), token

spot, sp_auth_token = auth_SpotPy()

In [13]:
# full albums from tracks in list (eg. release radar)

def createNewPL( pl_name="New_PL", stamp=True, ):
    try: return spot.user_playlist_create( 
        user= user_id, 
        name= f"{pl_name}_{ops.dtStamp()}" if stamp else pl_name, 
        public=False )
    except Exception as e: return f"EXC createNewPL:\n{type(e).__name__}\n{e}"

def albsFromPList( pl_id ):
    track_list = spot.user_playlist_tracks( user=user_id, playlist_id=pl_id, )
    return { i : {
        "alb_id" : item['track']['album']['id'], 
        "tracks" : item['track']['album']['total_tracks'],
        "dct" : item, } 
        for i, item in enumerate( track_list['items']) }

def getAllTrackIDs( listDict ):
    Alb = namedtuple( "Alb", [ "alb_id", "siz", "dct" ] )
    albsSorted = sorted( [ 
        Alb( d["alb_id"], d['tracks'], d["dct"] )
        for _, d in listDict.items() ],
        key = lambda el: el.siz, reverse=True )
    return [ item['id']
        for alb in albsSorted
        for item in spot.album_tracks( alb.alb_id )['items'] ]

def addTracksByID( idList, destination ):
    def yieldSegments(li, size):
        for i in range(0, len(li), size): yield li[ i:i + size ]
    for seg in yieldSegments(idList, 100):  # max 100
        spot.user_playlist_add_tracks(
            user_id, 
            playlist_id=destination, 
            tracks=seg)
            
created_pl = createNewPL( pl_name="rrad" )
rradDct = albsFromPList( "37i9dQZEVXbuX4MySjIacD" ) # release radar 
allTrackIDs = getAllTrackIDs( rradDct )
addTracksByID( allTrackIDs, created_pl['id'] )

In [145]:
# get DF all playlists, all tracks
def allPlaylistsDF():
    '''folder structure not available through API as at 221224'''
    incrmt = 50  # max 50
    offset = 0
    plists = {}
    while True:
        newItemDcts = spot.current_user_playlists( incrmt, offset)["items"]
        plists.update( { iDct["id"]: iDct for iDct in newItemDcts } )
        if len(newItemDcts) < incrmt: return pd.DataFrame( plists ).T
        else: offset += incrmt

def getAllTracks( id ):
    incrmt = 100  # max 100
    offset = 0
    tracks = {}
    while True:
        items = spot.user_playlist_tracks( 
            user=user_id, playlist_id=id, limit=incrmt, offset=offset )["items"]
        tracks.update( { i["track"]["id"]: i for i in items } )
        if len(items) < incrmt: return tracks
        else: offset += incrmt

# ~9min
df_pLists = allPlaylistsDF()
df_pLists["tracklist"] = df_pLists["id"].apply( lambda id: getAllTracks( id ) )
ops.storePKL( df_pLists, "df_pLists", os.getcwd() )